In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
import src.gen_source as gen_source
from guppy import hpy 
import mydab

import tcp_async
import tcp_sync

from live_analyse_py import live_analyse_py

In [ ]:
try:
    __IPYTHON__
    reload(tcp_async)
    reload(tcp_sync)
    reload(gen_source)
    reload(mydab)
except:
    pass

In [ ]:
sig = np.fromfile("/home/andreas/dab/dab_normalized_c64.dat", dtype=np.complex64)

#coefs = [0, -0.05, 0, 0, 0.05]
coefs = [0, 0, 0, 0, 0]
res = sig
for idx, coef in enumerate(coefs):
    res += sig * np.abs(sig)**(idx+1) * coef #+1 because first correction term is squared

res = res / np.max(res)

sig.tofile("./input.dat")
del res
del sig

In [ ]:
sync = tcp_sync.UhdSyncMsg(packet_size=4*8192,
                           packet_type="".join(["f"]*8192))
async = tcp_async.UhdAsyncMsg()

In [ ]:
top = live_analyse_py()

In [ ]:
samp_rate = 4000000 #samples / second
frequency_0  = samp_rate/10. #cycles / second
frequency_1  = samp_rate/9. #cycles / second
fft_size = 8192

In [ ]:
top.start()

In [ ]:
sync.has_msg()

In [ ]:
top.

In [ ]:
import peakutils
def score(msg, frequency_0, frequency_1, samp_rate):
    frequency_0_idx = np.round(frequency_0 / samp_rate * fft_size).astype(int)
    frequency_1_idx = np.round(frequency_1 / samp_rate * fft_size).astype(int)
    frequency_idx_diff = frequency_1_idx - frequency_0_idx
    
    start = frequency_0_idx % frequency_idx_diff + frequency_idx_diff / 2.
    start = np.round(start).astype(int)
    end   = msg.shape[0] - frequency_idx_diff - (msg.shape[0] - start) % frequency_idx_diff
    
    msg_cut = msg[start:end]
    
    msg_cut = msg_cut.reshape(-1, frequency_idx_diff)
    
    msg_peaks = np.max(msg_cut, axis=1)
    #plt.plot(msg_peaks, "ro")
    
    inter_mod = \
    (0.5 * (msg_peaks[8]/msg_peaks[7] + msg_peaks[9]/msg_peaks[10]),
     0.5 * (msg_peaks[8]/msg_peaks[6] + msg_peaks[9]/msg_peaks[11]),
     0.5 * (msg_peaks[8]/msg_peaks[5] + msg_peaks[9]/msg_peaks[12]),
     0.5 * (msg_peaks[8]/msg_peaks[4] + msg_peaks[9]/msg_peaks[13]),
     0.5 * (msg_peaks[8]/msg_peaks[3] + msg_peaks[9]/msg_peaks[14]),
     0.5 * (msg_peaks[8]/msg_peaks[2] + msg_peaks[9]/msg_peaks[15]),
     0.5 * (msg_peaks[8]/msg_peaks[1] + msg_peaks[9]/msg_peaks[16]))
    
    return(inter_mod)


In [ ]:
import itertools
x = np.linspace(-0.1, 0.1, num = 5)
l = list(itertools.product(x,x,x,x,x))
mag = [sum(map(lambda x:x*x,el)) for el in l]
idx = list(np.argsort(mag))
l = list(np.array(l)[idx])
len(l)

In [ ]:
top.set_txgain(84)
sync.has_msg()
msgs = np.array(sync.get_msgs(2))
fft = np.fft.fftshift(msgs[0])
#plt.plot(fft)
#plt.plot((4000 - f, 4000 -f),(-180,-40))

mydab.calc_signal_sholder_ratio(fft, fft_size, samp_rate)

In [ ]:
def meas(i):
    import sys
    sys.stdout.write(str(i) + " ")
    top.set_txgain(i)
    sync.has_msg()
    msgs = np.array(sync.get_msgs(10))
    
    res = []
    for msg in msgs:
        fft = np.fft.fftshift(msg)
        res.append(mydab.calc_signal_sholder_ratio(fft, fft_size, samp_rate))
    return np.mean(res)

In [ ]:
top.set_txgain(85)
mydab.calc_signal_sholder_ratio(fft, fft_size, samp_rate, plot=True)

In [ ]:
res = [meas(i) for i in range(89)]

In [ ]:
plt.plot(res)
plt.title("DAB sholders")#, coefs = [0, -0.05, 0, 0, 0.05]")
plt.xlabel("tx gain [db]")
plt.ylabel("signal - sholder [db]")

In [ ]:
sync.stop()
async.stop()
top.stop()
top.wait()